In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

df = pd.read_csv('data_karapetyan_final.csv', sep=';')

In [ ]:
null_cols=df.isnull().sum()
null_cols

In [ ]:
drop_percentage=round(null_cols[null_cols>0]/df.shape[0]*100,2)
drop_percentage

#percentage of null value in each column

In [ ]:
drop_cols=drop_percentage[drop_percentage>75].index
drop_cols

In [ ]:
df=df.drop(drop_cols,axis=1)

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.rename(columns= {"Name of the Movie":"Movie",
                    "Critics(CRS)":"Press Rating",
                    "IMDB":"IMDB Rating",
                    "Production Budget (PB)":"Budget",
                    'Box Office(US)':'US Gross'}, inplace=True)
df=df.drop(['Writ_Prize','Act_Nominated'],axis=1)
df.drop(list(df.filter(regex = 'Dir')), axis = 1, inplace = True)
df.head()

In [ ]:

df = df[pd.isnull(df['US Gross']) == False]
df.shape

#df = df.fillna(0)
#dg = df[df['US Gross'] != 0]

In [ ]:
import numpy as np
df['US Gross']=df['US Gross'].str.replace(" ","").astype(int)
df['Budget']=df['Budget'].str.replace(" ","").astype(int)
df['IMDB Rating']=df['IMDB Rating'].str.replace(" ","").str.replace(',','.').astype(np.float64)

df.dtypes

In [ ]:
sns.heatmap(df.corr(), cmap='YlGnBu')
#overview of the dataframe

#Gross is highly related to budget
#IMDB is highly related to Press
#Action type is highly related to Budget
#Animation type is highly related to PG
#Drama is mildly related to Press Rating
#Drama is mildly related to Actor Prized
#Drama is the most recognized genre
#Writer_Nom is higly related to Press Rating

In [ ]:

LG = df.drop(['Movie','G', 'PG', 'PG-13', 'R', 'Budget', 'US Gross'],axis=1)
sns.heatmap(LG.corr(), cmap='YlGnBu')

#zoom on correlation between 'Press & IMDB Ratings' and 'Genre & Awards recognition'

In [ ]:
plt.scatter(df.Gross, df.Budget)
plt.xlim(0, 250000000)
plt.ylim(0, 250000000)

In [ ]:
sns.lmplot('Gross','Budget',data=df)
plt.xlim(0, 300000000)
plt.ylim(0, 300000000)

# correlation budget / gross

In [ ]:
sns.lmplot('Press Rating','IMDB Rating',data=df)
#very strong correlation



In [ ]:
df['IMDB Rating'] = df['IMDB Rating'].apply(lambda x: x*10)
# converting IMDB ratings to the same scale of that in Press Ratings


In [ ]:
df[df['Press Rating']<20]
# three least liked movies by the press

In [ ]:
df[df['IMDB Rating']<45]
#three least liked movies by IMDB users
#IMDB users give "higher" low ratings to the movies they don't like
# than the press. 
#EMOJI is the only common movie in the 2 bottom three podiums.

In [ ]:
df['Average Rating'] = df[["IMDB Rating", "Press Rating"]].mean(axis=1)
#merging 2 columns into 1
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
ratings = df.ix[:, cols]
#bringing 'Average Rating to the beginning of the df for convenience purposes'
ratings=ratings.drop(['Movie','G', 'PG', 'PG-13', 'R', 'Budget', 'US Gross',
              "IMDB Rating", "Press Rating"],axis=1, inplace=True)

sns.heatmap(ratings.corr(), cmap='YlGnBu')
sns.jointplot('Average Rating','Drama', data = ratings, kind = 'kde')

#correlation analysis
#if a movie is a drama, it is more likely to get a high 
#average rating than if it is not.
# if a movie is a drama, chances are high that its writer and/or its main 
# actor got awards in the past.

In [ ]:
sns.jointplot('Average Rating','Action', data = ratings, kind = 'kde')
#action movies are overall mildly appreciated

In [ ]:
# animated movies
anim = df[['Animation','G','PG','PG-13','R']]
anim = anim[anim['Animation'] != 0]
anim.shape

#21 animated movies in the database

G = anim['G'].agg(sum)/anim['Animation'].agg(sum)*100
PG = anim['PG'].agg(sum)/anim['Animation'].agg(sum)
PG13= anim['PG-13'].agg(sum)/anim['Animation'].agg(sum)
R = anim['R'].agg(sum)/anim['Animation'].agg(sum)
animated = [G,PG,PG13,R]

def sd(x):
  return x*100  

percentages = [round((sd(x)),0) for x in animated]
percentages = [x for x in percentages if x != 0]
labels=['G','PG','R']
sizes=percentages
explode=(0.2,0.1,0.2)
fig1,ax1=plt.subplots()
ax1.pie(sizes,explode=explode,labels=labels,autopct='%1.1f%%',shadow=True,startangle=120)
plt.show()

# pie chart of Animated movies' parental ratings. PG indicates that 
# movies include some moments of mild violence. There's an evolution
# compared to the the big Disney movies that were all rated G.

In [ ]:
#PROFITABILITY

profit = df[['Movie','US Gross','Budget']]

profit['Profitability'] = ((profit['Gross']-profit.Budget)/profit.Budget)*100

profit.nsmallest(5, 'Profitability').sort_values('Profitability', ascending=False)
#the 5 least profitable movies of the list

In [ ]:
profit.nlargest(5, 'Profitability').sort_values('Profitability', ascending=False)
#the 5 most profitable movies of the list

In [ ]:
sns.distplot(profit.Profitability)
#overview of the profitability rates

In [ ]:
df[(df.Gross>300000000)&(df.Budget<90000000)]
 #one way to find very profitable movies.

In [ ]:
def label_category (row):
  if row['Budget'] >= 100000000 :
    return 'above $100M'
  elif 50000000 <= row['Budget'] < 100000000 :
    return 'above $50M'
  else :
      return 'below $50M'


profit['Budget League'] = profit.apply (lambda row: label_category(row), axis=1)

def label_profit (row):
  if row['Profitability'] >= 1000 :
    return 1000
  
  elif 500 <= row['Profitability'] < 1000 :
    return 500
  elif 200 <= row['Profitability'] < 500 :
    return 200
  elif 100 <= row['Profitability'] < 200 :
    return 100
  elif 0 <= row['Profitability'] < 100 :
    return 200
  elif -50 <= row['Profitability'] < 0 :
    return -50
  else :
      return -100


profit['profit'] = profit.apply (lambda row: label_profit(row), axis=1)

p = sns.FacetGrid(profit, col="Budget League")
p = p.map(plt.hist, "profit",bins=6)

#movies with a low budget are able to have crazy profitability rates,
# even if they aren't the ones that gross the most money.

In [ ]:
sequels = df[df.Movie.str.contains(':') |df.Movie.str.contains('\d', regex=True) |df.Movie.str.contains('\.')]
#creating dataframe focused on sequels



In [ ]:
sequels = sequels[(df.Movie.str.contains('50') == False)
                  & (df.Movie.str.contains('8') == False) & 
                  (df.Movie.str.contains('13') == False)]
# manual cleaning of the list because the number of rows 
# was low enough to make it possible

#sequels.shape
nonsequels = df[(df.Movie.str.contains(':') == False) 
                & (df.Movie.str.contains('\d', regex=True) == False) & 
                (df.Movie.str.contains('\.') == False)]

sequels_average_gross = sum(sequels['US Gross'])/len(sequels['US Gross'])
non_sequel_average_gross = sum(nonsequels['US Gross'])/len(nonsequels['US Gross'])

difference = sequels_average_gross - non_sequel_average_gross